In [202]:
import openai
import time
import pandas as pd

In [203]:
client = openai.Client()

Realiza o upload de um arquivo para carregar informações ao modelo

In [204]:
file = client.files.create(
    file=open("data_upload.txt", "rb"),
    purpose="assistants"
)

Cria o assistente para acessar o arquivo

In [205]:

assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas.",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o-mini"
)


Função: Cria uma thread de conversa e envia a pergunta como mensagem.

In [206]:
#pergunta = "Qual o rating médio das vendas do supermercado"
#pergunta = "Gere um grafico de vendas por linha de produtos"
pergunta = (
    "Gere 15 perguntas para conversação avançada em ingles que não estejam contidas no arquivo data_upload. "
    "Nao repita as questões e nem mostre o numero de sequencia delas"
)


# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta)

C:\Users\said-\AppData\Local\Temp\ipykernel_924\3607064502.py:10: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\said-\AppData\Local\Temp\ipykernel_924\3607064502.py:11: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [207]:
#Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

C:\Users\said-\AppData\Local\Temp\ipykernel_924\234429889.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [208]:
# Aguarda a thread rodar
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
        
    )
    print(run.status)

C:\Users\said-\AppData\Local\Temp\ipykernel_924\2396637559.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


in_progress
in_progress
in_progress
in_progress
in_progress
completed


In [209]:
# Verifica a resposta quando tivermos a resposta da thread
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens.data[0].content[0].text.value)
else:
    print(f"Erro {run.status}")

C:\Users\said-\AppData\Local\Temp\ipykernel_924\1247652421.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


Aqui estão 15 perguntas avançadas para conversação em inglês:

- How do you believe advancements in technology are reshaping human relationships?
- In your opinion, what are the most significant ethical dilemmas we face in the modern world?
- How do cultural narratives influence our perception of success and failure?
- If you could have dinner with any historical figure, who would it be and what question would you ask them?
- How do you think art and literature reflect societal values and challenges?
- What role does empathy play in resolving conflicts in both personal and global contexts?
- How can exposure to different cultures enhance our understanding of global issues?
- In what ways do you think social media has altered our perception of reality?
- How do you define personal growth, and what steps do you take to achieve it?
- How can the concepts of resilience and adaptability contribute to success in a rapidly changing job market?
- What are the implications of artificial intelli

In [210]:
# por hora esse print ficara commitado porque ja esta sendo considerado
# mais abaixo
# print(mensagens.data[0].content[0].text.value)

In [211]:
#analisando a logica/passos do modelo para chegar na resposta
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

C:\Users\said-\AppData\Local\Temp\ipykernel_924\3650132559.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [212]:
for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("-" * 10)
            print(tool_call.code_interpreter.input)
            print("-" * 10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
        thread_id=thread.id,
        message_id=step.step_details.message_creation.message_id
    )

# print(message.content[0].text.value)
# comentado acim apara quando a geração do output é grafico e não texto(conforme a pergunta)


===Step message_creation


C:\Users\said-\AppData\Local\Temp\ipykernel_924\1145170754.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


In [213]:

#condicional adaptado para quando a pergunta solicita texto ou grafico
#Função: Verifica se a resposta é texto ou imagem. Se for imagem, salva localmente.
if message.content[0].type == "text":
    print(message.content[0].text.value)
if message.content[0].type == "image_file":
    file_id = message.content[0].image_file.file_id
    image_data = client.files.content(file_id)
    with open(f"files/{file.id}.png", "wb") as f:
        f.write(image_data.read())
    print(f"Imagem {file_id} salva")


Aqui estão 15 perguntas avançadas para conversação em inglês:

- How do you believe advancements in technology are reshaping human relationships?
- In your opinion, what are the most significant ethical dilemmas we face in the modern world?
- How do cultural narratives influence our perception of success and failure?
- If you could have dinner with any historical figure, who would it be and what question would you ask them?
- How do you think art and literature reflect societal values and challenges?
- What role does empathy play in resolving conflicts in both personal and global contexts?
- How can exposure to different cultures enhance our understanding of global issues?
- In what ways do you think social media has altered our perception of reality?
- How do you define personal growth, and what steps do you take to achieve it?
- How can the concepts of resilience and adaptability contribute to success in a rapidly changing job market?
- What are the implications of artificial intelli